In [1]:
from src.rag.components.retriever import HybridRetriever

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [3]:
from pathlib import Path

In [4]:
model_path = Path.cwd().joinpath("models")

In [5]:
embedding_model_path = model_path.joinpath(embedding_model_id ).__str__()

In [6]:
cross_encoder_kwargs = {
    "model_name": embedding_model_path,
    "trust_remote_code": True,
    "local_files_only": True,
    "config_kwargs": {"use_memory_efficient_attention": False, "unpad_inputs": False},
    "device": "cpu"
}

In [7]:

transformer_kwargs  = { "model_name_or_path": embedding_model_path,
    "trust_remote_code": True,
    "device": "cpu",
    "config_kwargs": {"use_memory_efficient_attention": False,
                   "unpad_inputs": False},
    "cache_folder": model_path}

In [8]:
spacy_model_id = "en_core_web_sm"

In [9]:
retriever = HybridRetriever(
    cross_encoder_kwargs=cross_encoder_kwargs,
    spacy_model=spacy_model_id,
    sentence_transformer_kwargs=transformer_kwargs,
    language="english"
)

Some weights of the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of NewForSequenceClassification were not initialized from the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [54]:
query = "How does an interest only mortgage work?"


## "How can I speed up the mortgage completion process?"


#"How does an interest only mortgage work?"

# "How do you work out your LTV ratio?"

In [55]:
semantic_search_results = retriever.semantic_search(query=query, limit=10)

In [56]:
for item in semantic_search_results:
    print(item.content)
    print(10 * "-")

Use our mortgage calculator to work out how much you could borrow, and what your repayments might look like with an interest-only mortgage. Use our mortgage calculator
----------
Check out our interest-only mortgage FAQs for existing customers.
----------
An interest only mortgage covers the interest charges on your loan, rather than the original sum borrowed. This means you’ll still owe the lender the full amount you borrowed at the end of the term.
----------
What are mortgage interest rates?                                  expandable section
----------
Types of mortgages How does a mortgage work? Mortgage eligibility explained How to apply for a mortgage
----------
Free, impartial advice on interest-only mortgages from MoneyHelper.
----------
Pay off the mortgage interest, not the amount you borrowed, with an interest only mortgage.
----------
First Time Buyers Types of mortgage How to apply for a mortgage How does a mortgage work?
----------
Your monthly payment pays only the inte

In [57]:
keywords = retriever.perform_keyword_extraction(query)

In [58]:
keywords

'"mortgage work" or "interest"'

In [59]:
keywords_search_results = retriever.keyword_search(keywords=keywords, limit=10)

We can add a reranker but we decided to keep it simple for now.

For some reason the encoder is taking long to return.

In [60]:
for item in keywords_search_results:
    print(item.content)
    print(10 * "-")

Interest only Enter the amount you want on interest only. You can find your current balance through Online Banking or on your mortgage statement. Repayment Enter the amount you want on repayment. Term Interest only Enter your mortgage term to the nearest year. You can find this through Online Banking or on your mortgage statement. Repayment Enter your preferred mortgage term to the nearest year. You may be able to extend your term when you switch to a repayment mortgage to make it more affordable for you. This means you’ll pay less each month but pay more interest overall. You’ll need to meet our criteria to extend your term, and we can talk you through this when you get in touch.   Interest only Interest only Enter your interest rate e.g. 3.59%. You can find this through Online Banking or on your mortgage statement. Repayment Enter your interest rate e.g. 3.59%. We’ll tell you what rates you qualify for and are available when you apply. Monthly payment Interest only This is an estimat

In [36]:
# scores = retriever.cross_encoder.predict(
#    [(query, item.content) for item in keywords_search_results + semantic_search_results])

### Generator

In [37]:
from src.rag.components.generator import  LLamaCppGeneratorComponent

In [61]:
prompt = "you are a helpful mortgage advisor"

In [62]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [63]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [64]:
llama_cpp_generator._ping_api()

True

In [65]:
documents = [doc.content for doc in semantic_search_results +
             keywords_search_results]

In [66]:
documents

['Use our mortgage calculator to work out how much you could borrow, and what your repayments might look like with an interest-only mortgage. Use our mortgage calculator',
 'Check out our interest-only mortgage FAQs for existing customers.',
 'An interest only mortgage covers the interest charges on your loan, rather than the original sum borrowed. This means you’ll still owe the lender the full amount you borrowed at the end of the term.',
 'What are mortgage interest rates?                                  expandable section',
 'Types of mortgages How does a mortgage work? Mortgage eligibility explained How to apply for a mortgage',
 'Free, impartial advice on interest-only mortgages from MoneyHelper.',
 'Pay off the mortgage interest, not the amount you borrowed, with an interest only mortgage.',
 'First Time Buyers Types of mortgage How to apply for a mortgage How does a mortgage work?',
 "Your monthly payment pays only the interest charges on your loan – you don't pay off any of t

In [67]:
template_input = {
    "documents": documents,
    "question": query
}

In [68]:
llama_cpp_generator.generate_chat_input(template_values=template_input)

[{'role': 'system', 'content': 'you are a helpful mortgage advisor'},
 {'role': 'user',
  'content': "\n            DOCUMENTS:\n            \n             - Use our mortgage calculator to work out how much you could borrow, and what your repayments might look like with an interest-only mortgage. Use our mortgage calculator \n\n            \n             - Check out our interest-only mortgage FAQs for existing customers. \n\n            \n             - An interest only mortgage covers the interest charges on your loan, rather than the original sum borrowed. This means you’ll still owe the lender the full amount you borrowed at the end of the term. \n\n            \n             - What are mortgage interest rates?                                  expandable section \n\n            \n             - Types of mortgages How does a mortgage work? Mortgage eligibility explained How to apply for a mortgage \n\n            \n             - Free, impartial advice on interest-only mortgages from 

In [69]:
response = llama_cpp_generator.run(template_values=template_input)

In [70]:
print(response)

An interest only mortgage covers the interest charges on your loan, rather than the original sum borrowed. This means you'll still owe the lender the full amount you borrowed at the end of the term.

With an interest-only mortgage, you'll need to know from the start how you're going to find a lump sum to repay the loan at the end of the mortgage term. When you apply, we'll ask you to show us that you have a repayment plan(s) in place, to pay off everything that you owe at the end of the mortgage term.

Interest-only mortgages are only available when the loan amount is less than 75% of our latest valuation of the property.
